# Spark SQL

`SparkSession` is an entry point to programming Spark with the Dataset and DataFrame API. As of Spark 2.0, `SparkContext` is replaced by `SparkSession`. However, we are keeping the class here for backward compatibility.

To create a basic `SparkSession`, just use `SparkSession.builder`:

In [ ]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

In [ ]:
conf = SparkConf().setMaster('local[*]')
spark = SparkSession \
    .builder \
    .appName('Python Spark SQL basic example') \
    .config(conf=conf) \
    .getOrCreate()
    # .config('spark.sql.execution.arrow.pyspark.enabled', 'true')

In [ ]:
spark

You can also create the `SparkSession` from the existing `SparkContext`.

```python
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

conf = SparkConf() \
    .setAppName('Python Spark SQL basic example') \
    .setMaster('local[2]')
sc = SparkContext(conf=conf)
spark = SparkSession(sc)
```

We will cover only the basic command in this course as the Spark SQL will be mainly used to load the CSV file into the Spark DataFrame. Such DataFrame will be further used by the Spark MLlib in the next workshop. You can find the list of all features [here](http://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html).

# Data Sources

Spark SQL supports operating on a variety of data sources through the `DataFrame` interface. A `DataFrame` can be operated on using relational transformations and can also be used to create a temporary view. Registering a `DataFrame` as a temporary view allows you to run SQL queries over its data.

## Databases

Spark SQL includes a data source that can read data from other databases using JDBC. The results are returned as a DataFrame and they can easily be processed in Spark SQL or joined with other data sources. See more from https://spark.apache.org/docs/latest/sql-data-sources-jdbc.html

Here are some examples of how to read the data from SQL Server and PostgreSQL:
- SQL server
https://kontext.tech/column/spark/290/connect-to-sql-server-in-spark-pyspark
- PostgreSQL
https://stackoverflow.com/questions/34948296/using-pyspark-to-connect-to-postgresql

For the sake of simplicity, we will only cover the read data from a CSV file. It should be noted that the techniques we cover here can be applied to any data sources as long as they can be loaded into `DataFrame`.

## Local CSV files

We will use the `titanic_data_cleaner.csv` file from the previous lecture.

In [ ]:
# Load CSV files into Spark
df_csv = spark.read.csv(
    'titanic_data_cleaner.csv',
    header=True)

In [ ]:
# Show some rows
df_csv.show()

In [ ]:
# Show the schema
df_csv.printSchema()

In [ ]:
# Column names
df_csv.columns

You can return the contents of the Spark `DataFrame` as Pandas `pandas.DataFrame` via `toPandas()`.

**Caution**: The conversion takes time! Make sure you run on a small DataFrame.

In [ ]:
# Returns the contents of this DataFrame as Pandas pandas.DataFrame
# Cautions: the conversion takes time! --> make sure you run on 
df_csv.toPandas()

`select` projects a set of expressions and returns a new `DataFrame`.

In [ ]:
# Select only the 'Sex' column
df_csv.select('Sex').show()

In [ ]:
# Select only the 'SibSp' column and increase the 'SibSp' column by 1
df_csv.select(df_csv['SibSp'], df_csv['SibSp']+1).show()

In [ ]:
# Select the passengers who have SibSp>0
df_csv.filter(df_csv['SibSp'] > 0).show()

In [ ]:
# Number of samples per passenger class
df_csv.groupBy('Pclass').count().show()

In [ ]:
# Number of survived and died passengers
df_csv.groupBy('Survived').count().show()

The `sql` function on a `SparkSession` enables applications to run SQL queries programmatically and returns the result as a DataFrame.

In [ ]:
# Register the DataFrame as a table
df_csv.registerTempTable('titanic')

# Execute SQL query
df_sql = spark.sql('SELECT Age, Pclass, Fare, Survived FROM titanic')
df_sql.show()

# Built-in functions

Ref: http://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html#module-pyspark.sql.functions

In [ ]:
import pyspark.sql.functions as sql_fn

In [ ]:
# floor and ceil
df_csv.select(
    df_csv['Fare'],
    sql_fn.floor(df_csv['Fare']),
    sql_fn.ceil(df_csv['Fare'])).show()

In [ ]:
# Aggregate on the entire DataFrame without groups
df_csv.agg(
    sql_fn.count(df_csv['Fare']).alias('count_fare'),
    sql_fn.countDistinct(df_csv['Fare']),
    sql_fn.mean(df_csv['Fare']),
    sql_fn.sum(df_csv['Fare']),
    sql_fn.corr(df_csv['Fare'], df_csv['PClass'])).show()

In [ ]:
# Aggregate on the grouped data
df_csv.groupBy('Survived').agg(
    sql_fn.mean(df_csv['Fare']),
    sql_fn.sum(df_csv['Fare'])).show()